In [17]:
import pandas as pd
import re 
import nltk
nltk.download('punkt', download_dir='C:/Users/User/nltk_data')
nltk.download('punkt_tab', download_dir='C:/Users/User/nltk_data')
nltk.download('wordnet', download_dir='C:/Users/User/nltk_data')
nltk.download('omw-1.4', download_dir='C:/Users/User/nltk_data')
nltk.download('averaged_perceptron_tagger_eng', download_dir='C:/Users/User/nltk_data') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from collections import Counter
import numpy as np


[nltk_data] Downloading package punkt to C:/Users/User/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:/Users/User/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to C:/Users/User/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:/Users/User/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:/Users/User/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [19]:
df = pd.read_csv("text.csv")
df = df.head(5000)
if df.columns[0] == "Unnamed: 0":
    df.rename(columns={"Unnamed: 0": "index"}, inplace=True)
df.set_index("index", inplace= True)
print(df)

                                                    text  label
index                                                          
0          i just feel really helpless and heavy hearted      4
1      ive enjoyed being able to slouch about relax a...      0
2      i gave up my internship with the dmrg and am f...      4
3                             i dont know i feel so lost      0
4      i am a kindergarten teacher and i am thoroughl...      4
...                                                  ...    ...
4995       i long every moment to feel your gentle touch      2
4996   im just not feeling that needy or messed up an...      0
4997   i hate him and the feeling is pretty mutual i ...      3
4998   i can guarantee you that if i saw these jerks ...      4
4999   i know for a normal girl it makes her feel off...      3

[5000 rows x 2 columns]


In [20]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_sentence(sentence):
    sentence = sentence.split()
    tokens = word_tokenize(" ".join (sentence))
    pos_tags = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
    ]
    return ' '.join(lemmatized)

In [22]:
stopwords = {w.strip().lower() for w in open("stopwords.txt")}
tokens = []

def clean_text(text, stop_set=stopwords):
    row_tokens = re.findall(r"[a-zA-Z]+", text.lower())
    row_tokens = [w for w in row_tokens if w not in stop_set]
    tokens.extend(row_tokens)
    return " ".join(row_tokens)

def remove_ing(word):
    return re.sub(r'ing$', '', word)

df["lemmatized_text"] = df["text"].apply(lemmatize_sentence)
df["lemmatized_text"] = df["lemmatized_text"].apply(lambda sentence: " ".join(remove_ing(word) for word in sentence.split()))
df["lemmatized_text"] = df["lemmatized_text"].apply(clean_text)


In [24]:
print(df)

                                                    text  label  \
index                                                             
0          i just feel really helpless and heavy hearted      4   
1      ive enjoyed being able to slouch about relax a...      0   
2      i gave up my internship with the dmrg and am f...      4   
3                             i dont know i feel so lost      0   
4      i am a kindergarten teacher and i am thoroughl...      4   
...                                                  ...    ...   
4995       i long every moment to feel your gentle touch      2   
4996   im just not feeling that needy or messed up an...      0   
4997   i hate him and the feeling is pretty mutual i ...      3   
4998   i can guarantee you that if i saw these jerks ...      4   
4999   i know for a normal girl it makes her feel off...      3   

                                         lemmatized_text  
index                                                     
0         

In [ ]:
vc = pd.Series(tokens).value_counts()
vc = vc.head(7000)
words = list(vc.index.unique())
print(len(words))

In [ ]:
word_to_index = {}
index = 1 

for word in words:
    if word not in word_to_index:
        word_to_index[word] = index
        index += 1
print("Vocabulary:", word_to_index)


In [ ]:
new_words = text.split()
emotion_related_words = []
new_word = "feel"
emotion_related_words.extend(get_synonyms(word))
emotion_related_words.extend(get_antonyms(word))
emotion_related_words.extend(get_hyponyms(word))
emotion_related_words.extend(get_hypernyms(word))

print(set(get_synonyms(word)))

In [ ]:
# 1️⃣  Build / extend the vocabulary
special_tokens = ["<PAD>", "<UNK>", "<SOS>", "<EOS>"]

word_to_index = {}             # or load your existing dict here
for tok in special_tokens:      # make sure the specials are present first
    if tok not in word_to_index:
        word_to_index[tok] = len(word_to_index)

# Optionally add your corpus words afterwards
for sent in sentences:          # sentences is a list of token lists
    for word in sent:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

# 2️⃣  Keep the index of <UNK> handy
UNK_IDX = word_to_index["<UNK>"]

# 3️⃣  Robust mapping function
def sentence_to_sequence(sentence, word_to_index, unk_index=UNK_IDX):
    """
    Convert a list of tokens to their indices.
    Unknown words map to `unk_index`.
    """
    return [word_to_index.get(word, unk_index) for word in sentence]

# 4️⃣  Vectorise every sentence
sequences = [sentence_to_sequence(sent, word_to_index) for sent in sentences]
print("Sequences:", sequences[:10])


In [ ]:
all_words = []
for sentence in df['text']:
    all_words.extend(clean_text(sentence))

word_freq = Counter(all_words)

word2idx = {word: idx+2 for idx, word in enumerate(all_words)}

word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1

def text_to_sequence(text):
    return [word2idx.get(word, word2idx['<UNK>']) for word in clean_text(text)]


df['sequence'] = df['text'].apply(text_to_sequence)

In [ ]:
df['sequence']

In [ ]:

def pad_sequences(sequences, max_len):
    padded = np.zeros((len(sequences), max_len), dtype=int)
    for i, seq in enumerate(sequences):
        length = min(len(seq), max_len)
        padded[i, :length] = seq[:length]
    return padded

# Example: set max_len (you can use 50, 100, or based on your data)
max_len = 50  # try different values if needed

# Apply padding
padded_array = pad_sequences(df['sequence'], max_len)

In [ ]:
padded_array

In [ ]:
list_sadness = []
list_joy = []
list_love = []
list_anger = []
list_fear = []
list_surprise = []
list_unlabeled = []
def categorize_emotions(row):
    if row['label'] == 0:
        list_sadness.append(row['text'])
    elif row['label'] == 1:
        list_joy.append(row['text'])
    elif row['label'] == 2:
        list_love.append(row['text'])
    elif row['label'] == 3:
        list_anger.append(row['text'])
    elif row['label'] == 4:
        list_fear.append(row['text'])
    elif row['label'] == 5:
        list_surprise.append(row['text'])
    return row
df = df.apply(categorize_emotions, axis=1)

In [ ]:
sadness = pd.DataFrame({'text': list_sadness, 'label': 0})
joy = pd.DataFrame({'text': list_joy, 'label': 1})
love = pd.DataFrame({'text': list_love, 'label': 2})
anger = pd.DataFrame({'text': list_anger, 'label': 3})
fear = pd.DataFrame({'text': list_fear, 'label': 4})
surprise = pd.DataFrame({'text': list_surprise, 'label': 5})

In [ ]:
sorted_df = pd.concat([sadness, joy, love, anger, fear, surprise], ignore_index=True)


In [ ]:
sorted_df.head()

In [ ]:
type(df)

In [ ]:
sadness_tokens  = []
joy_tokens      = []
love_tokens     = []
anger_tokens    = []
fear_tokens     = []
surprise_tokens = []

def tokenize_sadness(row):
    token = row['text'].split()
    sadness_tokens.extend(token)
    return row
def tokenize_joy(row):
    token = row['text'].split()
    joy_tokens.extend(token)
    return row
def tokenize_love(row):
    token = row['text'].split()
    love_tokens.extend(token)
    return row
def tokenize_anger(row):
    token = row['text'].split()
    anger_tokens.extend(token)
    return row
def tokenize_fear(row):
    token = row['text'].split()
    fear_tokens.extend(token)
    return row
def tokenize_surprise(row):
    token = row['text'].split()
    surprise_tokens.extend(token)
    return row
sadness = sadness.apply(tokenize_sadness, axis=1)
joy = joy.apply(tokenize_joy, axis=1)
love = love.apply(tokenize_love, axis=1)
anger = anger.apply(tokenize_anger, axis=1)
fear = fear.apply(tokenize_fear, axis=1)
surprise = surprise.apply(tokenize_surprise, axis=1)


In [ ]:
sadness_words_vc  = pd.Series(sadness_tokens).value_counts()
joy_words_vc      = pd.Series(joy_tokens).value_counts()
love_words_vc     = pd.Series(love_tokens).value_counts()
anger_words_vc    = pd.Series(anger_tokens).value_counts()
fear_words_vc     = pd.Series(fear_tokens).value_counts()
surprise_words_vc = pd.Series(surprise_tokens).value_counts()

In [ ]:
sadness_words_vc.head(200)

In [ ]:
joy_words_vc.head(200)


In [ ]:
love_words_vc.head(50)


In [ ]:

anger_words_vc.head(50)


In [ ]:
fear_words_vc.head(200)


In [ ]:
surprise_words_vc.head(200)

In [ ]:
print("Sadness:",  len(sadness) / 416809 * 100)
print(" ")
print("joy:", len(joy) / 416809 * 100)
print(" ")
print("love:",  len(love) / 416809 * 100)
print(" ")
print("anger:",  len(anger) / 416809 * 100)
print(" ")
print("fear:",len(fear) / 416809 * 100)
print(" ")
print("surprise:",  len(surprise) / 416809 * 100)
print(" ")

In [ ]:
lemma = ""
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())
    return list(synonyms)

def get_antonyms(word):
    antonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            for ant in lemma.antonyms():
                antonyms.add(ant.name().lower())
    return list(antonyms)

def get_hypernyms(word):
    hypernyms = set()
    for syn in wn.synsets(word):
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                hypernyms.add(lemma.name().lower())
    return list(hypernyms)

def get_hyponyms(word):
    hyponyms = set()
    for syn in wn.synsets(word):
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                hyponyms.add(lemma.name().lower())
    return list(hyponyms)